In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from mediawiki import MediaWiki
from bs4 import BeautifulSoup
import string
from nltk import tokenize
from nltk.tag import StanfordNERTagger
import string
from nltk.tokenize import word_tokenize
import unicodedata
import timeit

In [3]:
# Using the Stanfors' default tagger
st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/stanford-ner.jar',
   encoding='utf-8') 

In [4]:
# Lists containing common prefixes and suffixes.
prefix = ['mr','by','mrs','miss','dr', 'president','prime minister']
# https://www.irfca.org/docs/place-names.html
#suffixes = ['nagar','colony','street','road','hill','river','temple']
#subs = ['pur','puram','ur','ghar','pura','ganj','abad','halli','keri']

In [5]:
#news_data = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/GDELT_2017/large_dataset/unrest_2017_ie_toi.csv', sep=',')
df = pd.read_csv('../document similarity/document_similarity_hatt.csv')

In [25]:
test_data = df.iloc[5].content

In [7]:
# Function for removing unnecessary characters.
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"=", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"/", " ", string)
    return string.strip()

In [9]:
import en_coref_lg
nlp = en_coref_lg.load()

In [10]:
preps_score={'"': 0.99,
 'down': 0.99,
 'across': 0.25,
 'near': 0.67,
 'inside': 0.23,
 'amid': 0.23,
 'in': 0.22,
 'from': 0.205,
 'off': 0.2,
 'at': 0.19,
 'like': 0.18,
 'outside': 0.12,
 'because': 0.1,
 'of': 0.09,
 'about': 0.08,
 'between': 0.075,
 'by': 0.07,
 'among': 0.07,
 'with': 0.07,
 'against': 0.07,
 'into': 0.06,
 'that': 0.05,
 'through': 0.045,
 'during':0.045,
 'after': 0.045,
 'on': 0.03,
 'while':0.03,
 'for': 0.02,
 'as': 0.015}

In [11]:
vocab = ["strike",
"unrest",
"masses",
"protest",
"demonstration",
"worker",
"union",
"company",
"caste",
"religious",
"ethnic",
"reformed",
"rebellion",
"defense",
"violence",
"war",
"armed",
"fight",
"Right",
"free",
"freedom",
"liberty",
"justice",
"Fair",
"unfair",
"unequal",
"terror",
"extreme",
"Bomb",
"IED",
"weapon",
"gun",
"wmd",
"threat",
"suicide",
"murder",
"Kill",
"death",
"explosive",
"military",
"police",
"elite",
"government",
"oppresive",
"power",
"regime",
"fraud",
"corruption",
"coup",
"safety",
"secure",
"insecure",
"protect",
"enemy",
"resist",
"hostage",
"truce",
"fire",
"greed",
"panic",
"inflation",
"Price",
"Food",
"Water"]

In [12]:
def get_whos(test_data):
    # INCLUDE NOUN PHRASES ALSO AS CANDIDATES OR CERTAIN WORDS LIKE GOVERNMENT.
    start = timeit.default_timer()
    # Convert the entire document data to sentences.
    text_sent = nltk.sent_tokenize(test_data)
    stanford = []
    noun_phrases = []
    prp_phrases = []
    #new_model = []
    whos = []
    # other are prefixes
    other = []
    #suffix_locations = []
    #sub_locations = []
    all_words = []
    # Iterate over all the sentences.
    sent_no=0;
    total_words = 0;
    for sent in text_sent:
        # Convert the sentence to words
        text_tags = nltk.word_tokenize(sent)
        curr_words = len(text_tags)
        all_words+=[text_tags]
        # Get the NER for the words.
        original_tags = st.tag(text_tags)
        l = len(original_tags)
        i=0;
        # Iterate over the tagged words.
        while i<l:
            #print(i)
            e,t = original_tags[i];
            ci = i;
            # If it's a location, then check the next 3 words.
            if t == 'PERSON' or t == 'ORGANIZATION':
                j = 1;
                s = e; 
                # Verify the tags for the next 3 words.
                while i+j<len(original_tags):
                    # If the next words are also locations, then concatenate them to make a large string.
                    if original_tags[i+j][1] == 'PERSON' or original_tags[i+j][1] == 'ORGANIZATION':
                        s = s+" "+original_tags[i+j][0];
                        j+=1;
                    else:
                        break;
                i = i+j;
                # Save the candidates to a whos list
                whos+=[(sent_no,total_words+ci,s,ci)];
            else:
                i=i+1;
        # Get the POS tag for the words.
        text_tagged = nltk.pos_tag(text_tags)
        l = len(text_tagged)
        i=0;
        # iterate over the tagged words.
        while i<l:
            #print(i)
            e,t = text_tagged[i];
            # If the current word is a Noun phrase.
            if t == 'NNP':
                j = 1;
                s = e;
                # verify the tags for the next 3 words.
                while i+j<len(text_tagged):
                    # If the next words are also nouns, then concatenate them to make a large string.
                    if text_tagged[i+j][1] in ['NNP','NN','NNS','NNPS']:
                        s = s+" "+text_tagged[i+j][0];
                        j+=1;
                    else:
                        break;
                i = i+j;
                # Save the noun phrases to a noun_phrases list
                noun_phrases+=[s];
            elif t == 'PRP' or t == 'PRP$' or t == 'WP' or t == 'WP$':
                j = 1;
                s = e;
                # verify the tags for the next 3 words.
                while i+j<len(text_tagged):
                    # If the next words are also nouns, then concatenate them to make a large string.
                    if text_tagged[i+j][1] in ['PRP','PRP$','WP','WP$']:
                        s = s+" "+text_tagged[i+j][0];
                        j+=1;
                    else:
                        break;
                i = i+j;
                # Save the noun phrases to a noun_phrases list
                prp_phrases+=[s];
            else:
                i=i+1;
        # Convert the words to lower case so as to compare with the prefixes.
        text_tags = [t.lower() for t in text_tags]
        # Go through all the prefixes.
        for k in prefix:
            # If the prefix is not present in the sentence then an error occurs. So, we use a try, catch block
            try:
                # Get the position the prefix word in a sentence.
                index = text_tags.index(k)
                # If the word after the prefix word is a noun, then consider it.
                if text_tagged[index+1][1] == 'NNP':
                    j = 1;
                    s = text_tagged[index+1][0]; 
                    # Verify if it is a phrase by considering the next 3 words.
                    while index+j<len(text_tagged):
                        if text_tagged[index+j+1][1] in ['NNP','NN','NNS','NNPS']:
                            s = s+" "+text_tagged[index+j+1][0];
                            j+=1;
                        else:
                            break;
                    # Save the words after the prefix words in to other list.
                    other = other + [(sent_no,index+1,s)]
            except:
                continue
        sent_no+=1;
        total_words+=curr_words
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    all_words = [x for sublist in all_words for x in sublist]
    #only by stanford ner
    return all_words, whos, text_sent

In [13]:
def get_unrest_whos(all_words, whos):
    pos_threshold = len(all_words)
    all_whos = {k[2]:[] for k in whos}
    #nearest_whos = {k[2]:[] for k in whos}
    pos_occurences = {k[2]:[] for k in whos}
    pos_scores = {k:-1 for k in pos_occurences.keys()}
    unrest_scores = {k:-1 for k in all_whos.keys()}
    for location in list(all_whos.keys()):
        sub_locations = location.split(' ')
        if len(sub_locations) > 1:
            last_subloc = sub_locations[-1]
            first_subloc = sub_locations[0]
            first_indices = [i for i, w in enumerate(all_words) if w == first_subloc]
            last_indices = [i for i, w in enumerate(all_words) if w == last_subloc]
            diff = [a_i - b_i for a_i, b_i in zip(last_indices, first_indices)]
            first_indices = first_indices[diff == len(sub_locations)]
            if not isinstance(first_indices,list):
                pos_occurences[location].append(first_indices)
            else:
                pos_occurences[location] = first_indices
            indices = pos_occurences[location]
        else:
            first_indices = [i for i, w in enumerate(all_words) if w == sub_locations[0]]
            if not isinstance(first_indices,list):
                pos_occurences[location].append(first_indices)
            else:
                pos_occurences[location] = first_indices
            indices = pos_occurences[location]
        print(location)
        print(indices)
        if len(indices)>0:
            no_of_indices = len(indices)
            i=0
            b=[]
            #nearest_whos = []
            while i< no_of_indices:
                j=0
                a = []
                l = []
                while j<pos_threshold and indices[i]+j<len(all_words):
                    if all_words[indices[i]+j].lower() in vocab:
                        s = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                        a.append(s)
                    if all_words[indices[i]+j] in all_whos:
                        lo = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                        l.append(lo)
                    j=j+1
                j=0
                while j<pos_threshold and indices[i]-j>=0:
                    if all_words[indices[i]-j].lower() in vocab:
                        s = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                        a.append(s)
                    if all_words[indices[i]-j] in all_whos:
                        lo = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                        l.append(lo)
                    j=j+1
                i=i+1
                b = b+a
                #nearest_whos = nearest_whos+l
            all_whos[location] = b
    return all_whos

In [14]:
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [15]:
def get_modified_whos(all_whos):
    i=0;
    whos = {}
    while i<len(all_whos):
        if all_whos[i][2] not in whos:
            whos[all_whos[i][2]] = [(all_whos[i][0],all_whos[i][1],all_whos[i][3])]
        j=i+1;
        while j < len(all_whos):
            if all_whos[j][2].lower() == all_whos[i][2].lower():
                whos[all_whos[i][2]]+=[(all_whos[j][0],all_whos[j][1],all_whos[j][3])]
                all_whos.pop(j);
                #do not increment j
                continue;
            if findWholeWord(all_whos[j][2].lower())(all_whos[i][2].lower()) or findWholeWord(all_whos[i][2].lower())(all_whos[j][2].lower()):
                whos[all_whos[i][2]]+=[(all_whos[j][0],all_whos[j][1],all_whos[j][3])]
                all_whos.pop(j);
                continue;
            j+=1;
        i+=1
    return whos

In [16]:
#avg is considered
def get_pos_sent_scores(modified_who_candidates, all_words, text_sent):
    pos_scores = {k:0 for k in modified_who_candidates.keys()}
    sent_scores = {k:0 for k in modified_who_candidates.keys()}
    l = len(all_words)
    n = len(text_sent)
    for s in modified_who_candidates.keys():
        pos = modified_who_candidates[s];
        pos_score = 0;
        sent_score = 0;
        for p in pos:
            pos_score+=(1 - p[1]/l)
            sent_score+=(1 - p[0]/n)
        pos_score = pos_score/len(pos)
        sent_score = sent_score/len(pos)
        pos_scores[s] = pos_score
        sent_scores[s] = sent_score

    return pos_scores,sent_scores

In [17]:
def get_modified_unrest_whos(all_whos):
    i=0;
    whos = {}
    who_keys = sorted(all_whos, key=len, reverse=True)
    while i<len(who_keys):
        if who_keys[i] not in whos:
            print(who_keys[i])
            print('-----------')
            whos[who_keys[i]] = all_whos[who_keys[i]]
        j=i+1;
        #t = [];
        while j < len(who_keys):
            #print(who_keys[i])
            if who_keys[i].lower() == who_keys[j].lower():
                whos[who_keys[i]].extend(all_whos[who_keys[j]])
                who_keys.pop(j);
                #do not increment j
                continue;
            if findWholeWord(who_keys[i].lower())(who_keys[j].lower()) or findWholeWord(who_keys[j].lower())(who_keys[i].lower()):
                whos[who_keys[i]].extend(all_whos[who_keys[j]])
                print(who_keys[i])
                print(who_keys[j])
                who_keys.pop(j);
                continue;
            j+=1;
        #if(len(t) !=0):
        #    print('adding....')
        #print(t)
        #whos[who_keys[i]].extend(t)
        i+=1
    return whos

In [18]:
def get_title_score(whos, title, ):
    title_score = {k:0 for k in whos.keys()}
    title_words = nltk.word_tokenize(title)
    for k in title_score.keys():
        for w in title_words:
            if k.lower() == w.lower():
                title_score[k] = 1;
            if findWholeWord(k.lower())(w.lower()) or findWholeWord(w.lower())(k.lower()):
                print(k)
                print(w)
                title_score[k] = 1;
    return title_score

In [19]:
def get_unrest_scores(modified_unrest_whos):
    thresh = 10;
    unrest_scores = {k:-1 for k in modified_unrest_whos.keys()}
    for l in modified_unrest_whos.keys():
        score =0;
        if len(modified_unrest_whos[l]) >0:
            locs_unrest = modified_unrest_whos[l]
            for l2 in locs_unrest:
                if l2[0]<thresh:
                    score = score+(1/l2[0]);  
                else:
                    score = score+np.exp(-l2[0]); 
        unrest_scores[l] = score/len(modified_unrest_whos);
    return unrest_scores

In [20]:
def get_frequency(modified_who_candidates):
    frequecies = {k:-1 for k in modified_who_candidates.keys()}
    for l in modified_who_candidates.keys:
        frequecies[l] = len(modified_who_candidates[l])
    return frequencies

In [21]:
def get_sent_nos(modified_who_candidates):
    sent_nos = {k:[] for k in modified_who_candidates.keys()}
    for k in modified_who_candidates.keys():
        a = modified_who_candidates[k]
        c = []
        for b in a:
            c.extend([b[0]])
        sent_nos[k] = set(c)
    return sent_nos

In [22]:
def get_preps_scores(modified_who_candidates,all_words,preps_score):
    preps_scores = {k:[] for k in modified_who_candidates.keys()}
    for name in modified_who_candidates.keys():
        indices = modified_who_candidates[name]
        s = []
        for tup in indices:
            i=tup[1]
            try:
                if i-1>0 and all_words[i-1] in preps_score.keys():
                    s.append(preps_score[all_words[i-1]])
                elif i-2>0 and all_words[i-1] in prefix and all_words[i-1] in preps_score.keys():
                    s.append(preps_score[all_words[i-2]])
                else:
                    s.append(0);   
            except:
                    continue
        # Taking an average
        preps_scores[name] = np.average(s)
    # preps_scores: Scores to each location mention in an article based on the prepositions preceding them.
    return preps_scores

In [23]:
test_data = unicodedata.normalize("NFKD", test_data)
test_data = clean_str(test_data)
doc = nlp(test_data)
test_data = doc._.coref_resolved

In [22]:
 all_words, whos, text_sent = get_whos(test_data)

Time:  30.990341798999907


In [23]:
unrest_whos = get_unrest_whos(all_words,whos)

Anna Hazare
[0]
Jan Lokpal Bill Veteran Gandhian
[7]
Jan Lokpal Bill
[7]
Mahatma Gandhi
[62]
Swami Agnivesh
[76]
Kiran Bedi
[85]
Sandeep Pandey
[92]
Hazare
[1, 15, 131, 148, 184, 203, 234, 237, 240, 296, 300, 400]
Prime Minister s Office
[44]
Bill
[9, 34, 127, 285]
Santosh Hegde
[314]
Prashant Bhushan
[320]
‘ India Against Corruption
[341]


In [24]:
modified_who_candidates = get_modified_whos(whos)

In [25]:
modifies_unrest_whos = get_modified_unrest_whos(unrest_whos)

Jan Lokpal Bill Veteran Gandhian
-----------
Jan Lokpal Bill Veteran Gandhian
Jan Lokpal Bill
Jan Lokpal Bill Veteran Gandhian
Bill
‘ India Against Corruption
-----------
Prime Minister s Office
-----------
Prashant Bhushan
-----------
Mahatma Gandhi
-----------
Swami Agnivesh
-----------
Sandeep Pandey
-----------
Santosh Hegde
-----------
Anna Hazare
-----------
Anna Hazare
Hazare
Kiran Bedi
-----------


In [26]:
pos_scores,sent_scores=get_pos_sent_scores(modified_who_candidates,all_words,text_sent)

In [27]:
title_scores = get_title_score(modified_who_candidates,"Anna Hazare on fast unto death demanding Jan Lokpal Bill")

Anna Hazare
Anna
Anna Hazare
Hazare
Jan Lokpal Bill Veteran Gandhian
Jan
Jan Lokpal Bill Veteran Gandhian
Lokpal
Jan Lokpal Bill Veteran Gandhian
Bill


In [28]:
unrest_scores = get_unrest_scores(modifies_unrest_whos)

In [29]:
sent_nos = get_sent_nos(modified_who_candidates)

In [30]:
preps_scores = get_preps_scores(modified_who_candidates,all_words,preps_score)

In [31]:
def get_final_scores(pos_scores,sent_scores,unrest_scores,preps_scores,title_scores):
    final_scores = {k:-1 for k in pos_scores.keys()}
    for i in final_scores.keys():
            score = pos_scores[i]+sent_scores[i]+unrest_scores[i]+preps_scores[i]+title_scores[i]
            final_scores[i] = score/5;
    # final_scores: A combination of all the above scores for a location.
    return final_scores

In [32]:
final_scores = get_final_scores(pos_scores,sent_scores,unrest_scores,preps_scores,title_scores)

In [114]:
# Generate dependency trees for each of the sentence in which a who candidate appears.
import os
from nltk.parse.stanford import StanfordDependencyParser
from graphviz import Source

# make sure nltk can find stanford-parser
# please check your stanford-parser version from brew output (in my case 3.6.0) 
os.environ['CLASSPATH'] = r'/usr/local/Cellar/stanford-parser/3.9.1/libexec'

In [115]:
for i in modified_who_candidates.keys():
    tups = modified_who_candidates[i]
    for j in tups:
        sent_no = j[0]
        sentence = text_sent[sent_no]
        sdp = StanfordDependencyParser()
        result = list(sdp.raw_parse(sentence))

        dep_tree_dot_repr = [parse for parse in result][0].to_dot()
        source = Source(dep_tree_dot_repr, filename=i+"-"+str(sent_no), format="png")
        source.view(filename=i+"-"+str(sent_no),directory='dep graphs/')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  


KeyboardInterrupt: 

In [2]:
import tensorflow_hub as hub
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")
def run_gse_benchmark(sentence1, sentence2):
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
    sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))
        
    sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
      
        [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: sentence1,
                sts_input2: sentence2
            })
    return gse_sims

In [1]:
#Infer sent
import torch
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'dataset/GloVe/glove.840B.300d.txt' #if model_version == 1 else '../dataset/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
def run_infer_sent(sentence1,sentence2):
    return cosine(model.encode([sentence1],tokenize=True)[0], model.encode([sentence2],tokenize=True)[0])

Vocab size : 100000


In [33]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
def get_verb_similarity(phrase):
    words = nltk.word_tokenize(phrase)
    for w in words:
        if lmtzr.lemmatize(w) in vocab:
            return 1;
    sent_sim = 0;
    for w in words:
        for v in vocab:
            #s = run_gse_benchmark([w],[v])
            s = run_infer_sent(w,v)
            if s > sent_sim:
                sent_sim = s
    return sent_sim

In [34]:
def get_who_index(who,p1):
    words = nltk.word_tokenize(who)
    for w in words:
        for i in range(0,len(p1.nodes)):
            if p1.nodes[i]['word'] != None and p1.nodes[i]['word'].lower() == w.lower():
                return p1.nodes[i]['address']
    return -1

In [35]:
def get_who_root(ind,p1):
    for i in range(0,len(p1.nodes)):
        dependencies = p1.nodes[i]['deps'].keys()
        for d in dependencies:
            if ind in p1.nodes[i]['deps'][d]:
                return p1.nodes[i]['address']
    return -1

In [4]:
common_Dependencies = ['nsubj','csubj','dobj','iobj','obj','subj','dep']

In [5]:
def build_phrase(ind,p1):
    phrase = []
    phrase.append(p1.root['address'])
    phrase.append(p1.nodes[ind]['address'])
    other_deps = p1.nodes[ind]['deps'].keys()
    for d in other_deps:
        for i in p1.nodes[ind]['deps'][d]:
            #considering all the right siblings
            if i > ind:
                for cd in common_Dependencies:
                    if d in cd:
                        phrase.append(i)
    phrase.append(get_right_node(max(phrase),p1))
    return sorted(list(set(phrase)))

In [21]:
def get_right_node(ind,p1):
    node = p1.nodes[ind]
    while(len(node['deps'])!=0):
        a= []
        for i in node['deps'].values():
            a = a+i
        ind = max(a)
        node = p1.nodes[ind]
    return ind

In [47]:
# Generate dependency trees for each of the sentence in which a who candidate appears.
import os
from nltk.parse.stanford import StanfordDependencyParser
from graphviz import Source

# make sure nltk can find stanford-parser
# please check your stanford-parser version from brew output (in my case 3.6.0) 
os.environ['CLASSPATH'] = r'/usr/local/Cellar/stanford-parser/3.9.1/libexec'

In [49]:
def get_what_phrase(who,who_sent_number,text_sent,all_words):
    sdp = StanfordDependencyParser()
    result = sdp.raw_parse(text_sent[who_sent_number])
    p1 = 0
    for p in result:
        p1 = p
    ind = get_who_index(who,p1)
    ind = get_who_root(ind,p1)
    li = build_phrase(ind,p1)
    phr = ""
    for i in range(min(li),max(li)+1):
        phr = phr+all_words[i]+" "
    return phr.strip()

In [52]:
def get_what_candidates(modified_who_candidates,text_sent,all_words):
    what_candidates = {}
    for w in modified_who_candidates.keys():
        for t in modified_who_candidates[w]:
            p = get_what_phrase(w,t[0],text_sent,all_words)
            what_candidates[p] = t[0]
    return what_candidates

In [39]:
def get_what_sim_scores(what_candidates):
    sim_scores = {k:0 for k in what_candidates.keys()}
    for w in what_candidates.keys():
        sim_scores[w] = get_verb_similarity(w)
    return sim_scores

In [64]:
# provide best who and where sent  numebers as input
def get_distance_to_where_who(what_candidates, best_who, best_where,text_sent):
    dist_scores = {k:0 for k in what_candidates.keys()}
    for d in dist_scores.keys():
        dist_scores[d] = ((1 - abs(what_candidates[d] - best_who)/len(text_sent)) + (1 - abs(what_candidates[d] - best_where)/len(text_sent)))/2
    return dist_scores

In [53]:
# get what candidates
# get what sim scores
# get what distance to best where, who scores
what_candidates = get_what_candidates(modified_who_candidates,text_sent,all_words)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  
/anaconda3

In [54]:
what_candidates

{', Mr Anna Hazare , began a': 0,
 ', Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 12,
 'Bill Veteran Gandhian': 5,
 'Gandhian , Mr Anna Hazare , began a fast unto death in': 5,
 'Hazare , began a fast unto death in the Capital on Tuesday demanding': 1,
 'Lokpal Bill Veteran Gandhian ,': 12,
 'Lokpal Bill to check corruption .': 3,
 'Mr Anna Hazare , began a': 4,
 'corruption . Ignoring an': 3,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto': 2,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 2,
 'demanding Jan Lokpal Bill Veteran Gandhian ,': 6,
 'fast unto': 9,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began': 10,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto death in the': 10,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian ,

In [55]:
sim_scores = get_what_sim_scores(what_candidates)

In [56]:
sim_scores

{', Mr Anna Hazare , began a': 0.85450405,
 ', Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 1,
 'Bill Veteran Gandhian': 0.99999994,
 'Gandhian , Mr Anna Hazare , began a fast unto death in': 1,
 'Hazare , began a fast unto death in the Capital on Tuesday demanding': 1,
 'Lokpal Bill Veteran Gandhian ,': 0.99999994,
 'Lokpal Bill to check corruption .': 1,
 'Mr Anna Hazare , began a': 0.85450405,
 'corruption . Ignoring an': 1,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto': 1,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 1,
 'demanding Jan Lokpal Bill Veteran Gandhian ,': 0.99999994,
 'fast unto': 0.6805536,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began': 1,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto death in the': 1,
 'fast unto d

In [66]:
#this is a test
dist_scores = get_distance_to_where_who(what_candidates,2,3,text_sent)

In [67]:
dist_scores

{', Mr Anna Hazare , began a': 0.8076923076923077,
 ', Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 0.2692307692307692,
 'Bill Veteran Gandhian': 0.8076923076923077,
 'Gandhian , Mr Anna Hazare , began a fast unto death in': 0.8076923076923077,
 'Hazare , began a fast unto death in the Capital on Tuesday demanding': 0.8846153846153846,
 'Lokpal Bill Veteran Gandhian ,': 0.2692307692307692,
 'Lokpal Bill to check corruption .': 0.9615384615384616,
 'Mr Anna Hazare , began a': 0.8846153846153846,
 'corruption . Ignoring an': 0.9615384615384616,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto': 0.9615384615384616,
 'death demanding Jan Lokpal Bill Veteran Gandhian , Mr Anna Hazare , began a fast unto death in the Capital on Tuesday demanding': 0.9615384615384616,
 'demanding Jan Lokpal Bill Veteran Gandhian ,': 0.7307692307692307,
 'fast unto': 0.5,
 'fast unto death demanding Jan Lokpal Bill Veteran Gandhian , Mr A

In [26]:
test_data

'Jan Lokpal Bill: Protests continue in support of Anna Hazare in Punjab\n\nCHANDIGARH: Khap panchayats today joined the protests in support of Anna Hazare in Punjab and Haryana where the Gandhian\'s supporters took out car and bike rallies and staged a demonstration outside Prime Minister Manmohan Singh\'s private residence here.\xa0\n\nAs the protests spread, people in rural areas were seen participating in demonstrations in support of Hazare.\xa0\n\nKhap spokesman Suresh Koth said the khap panchayats (caste councils) in Haryana are observing a hunger strike in Jind, the Jat heartland of Haryana, and other places in support of Hazare\'s movement.\xa0\n\nIn Chandigarh, activists of various organisations staged a protest outside the Prime Minister\'s private residence in Sector 11.\xa0\n\nThe Chandigarh administration had yesterday promulgated prohibitory orders banning assembly of five or more persons outside the residences of five VIPs, including that of the Prime Minister.\xa0\n\nThe

In [17]:
import numpy as np
max(np.array([[1,2],[3,4,5]]).ravel())

[3, 4, 5]